# Modelo de Detección de Fraude

Durante el siguiente script se detallan e implementan los pasos para la creación de un modelo de detección de fraude a partir de características de cada una de las transacciones registradas.

Se decidió usar el modulo de Spark integrado a python ya que cuenta con una capacidad para manejar grandes volumenes de datos a diferencia de librerias como pandas. Además de contar con modelos pre-cargados con funciones que ayudan a su entrenamiento y evaluación de manera más amigable, concisa y rapida como veremos más adelante.

In [1]:
from pyspark.sql import SparkSession

# Creamos una sesión de Spark
spark = SparkSession.builder \
    .appName("FraudDetection") \
    .getOrCreate()

# Exploratory Data Analysis (EDA)

Nos encargamos de cargar los datos y vemos en que consisten y que columnas podrán ayudarnos a alimentar el modelo y cumplir nuestro objetivo.

In [2]:
PATH = 'datos_fraud.csv' #definimos la ruta al archivo, en este caso lo tenemos en la misma carpeta.

# Se cargan los datos y se muestran las primeras filas del df (DataFrame)
df = spark.read.csv(PATH, header=True, inferSchema=True) #como el .csv cuenta con encabezado lo acalaramos.
df.show()

print(f"numero de muestras: {df.count()}")

+--------------------+---------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+--------------------+-------------------+--------------------+-------------------+-------------------+--------------------+--------+
|      transaction_id|timestamp|amount|         variable_01|         variable_02|         variable_03|         variable_04|         variable_05|         variable_06|         variable_07|         variable_08|         variable_09|         variable_10|         variable_11|         variabl

In [3]:
# Vemos los tipos de datos de cada una de las columnas del dataframe
df.printSchema()

root
 |-- transaction_id: string (nullable = true)
 |-- timestamp: double (nullable = true)
 |-- amount: double (nullable = true)
 |-- variable_01: double (nullable = true)
 |-- variable_02: double (nullable = true)
 |-- variable_03: double (nullable = true)
 |-- variable_04: double (nullable = true)
 |-- variable_05: double (nullable = true)
 |-- variable_06: double (nullable = true)
 |-- variable_07: double (nullable = true)
 |-- variable_08: double (nullable = true)
 |-- variable_09: double (nullable = true)
 |-- variable_10: double (nullable = true)
 |-- variable_11: double (nullable = true)
 |-- variable_12: double (nullable = true)
 |-- variable_13: double (nullable = true)
 |-- variable_14: double (nullable = true)
 |-- variable_15: double (nullable = true)
 |-- variable_16: double (nullable = true)
 |-- variable_17: double (nullable = true)
 |-- variable_18: double (nullable = true)
 |-- variable_19: double (nullable = true)
 |-- variable_20: double (nullable = true)
 |-- varia

Como vimos en el output anterior, el dataframe esta compuesto por cinco tipos de columnas, de las cuales:

- transaction_id: se trata de un identificador que nos ayuda a diferencias cada conjunto de caracteristicas asociado a un registro
- time_stamp: para nuestro modelo, podemos omitir esta columna pues la hora y fecha en la que se realizo la transacción no afecta necesariamente para saber si es o no un fraude.
- amount: la cantidad de credito que se maneje puede ser importante para el analisis, pues a una cantidad mayor de monto podemos interpretar que podría existir mayor riesgo de fraude.
- variables 1  a la 32: variables propias de la transaccion que nos ayudan directamente como entrada para nuestro modelo. Al ser un ejercicio el significado independiente de cada una de estas variables no es de importancia.
- is_fraud: es la etiqueta con la que sabremos si la transacción es un fraude o no.

In [4]:
# Ver estadísticas descriptivas
df.describe().show()

+-------+--------------------+-----------------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|summary|      transaction_id|        timestamp|            amount|         variable_01|         variable_02|         variable_03|         variable_04|         variable_05|         variable_06|         variable_07|         variable_08|    

In [5]:
# Verificamos si existen valores nulos
from pyspark.sql.functions import col, isnan, when, count

#No cuenta con valores nulos
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+--------------+---------+------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+--------+
|transaction_id|timestamp|amount|variable_01|variable_02|variable_03|variable_04|variable_05|variable_06|variable_07|variable_08|variable_09|variable_10|variable_11|variable_12|variable_13|variable_14|variable_15|variable_16|variable_17|variable_18|variable_19|variable_20|variable_21|variable_22|variable_23|variable_24|variable_25|variable_26|variable_27|variable_28|variable_29|variable_30|variable_31|variable_32|is_fraud|
+--------------+---------+------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----

Para solucionar el problema del desbalanceo en los datos, se realiza un submuestreo.

In [6]:
# Submuestreo aleatorio de la clase mayoritaria (transacciones legítimas)
n_fraud = df.filter(col('is_fraud') == 1).count()
n_legit = df.filter(col('is_fraud') == 0).count()

df_legit_subsampled = df.filter(col('is_fraud') == 0).sample(False, n_fraud / n_legit, seed=42)

# Filtrar las transacciones fraudulentas
df_fraud = df.filter(col('is_fraud') == 1)

# Combinar el subconjunto de transacciones legítimas submuestreado con las transacciones fraudulentas
df = df_legit_subsampled.union(df_fraud)

In [7]:
# Ver estadísticas descriptivas. Ahora el promedio del valor is_fraud es mas cercano a 0.5
df.describe().show()
print(f"numero de muestras: {df.count()}")

+-------+--------------------+-----------------+------------------+--------------------+-------------------+--------------------+-------------------+--------------------+--------------------+--------------------+-------------------+-------------------+------------------+-------------------+-------------------+------------------+--------------------+-------------------+--------------------+-------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+------------------+------------------+------------------+
|summary|      transaction_id|        timestamp|            amount|         variable_01|        variable_02|         variable_03|        variable_04|         variable_05|         variable_06|         variable_07|        variable_08|        variable_09|       variable_10|    

Como vemos, el paso de limpieza de datos podemos omitirlo, ya que nuestro dataset no cuenta con datos nulos o incorrectos. 

Para evitar la predominancia de alguna de las caracteristicas en la predicción del modelo, se opta por realizar la normalización de los datos de entrada.

In [8]:
from pyspark.ml.feature import VectorAssembler, MinMaxScaler

#Convertimos las características en un vector, esto para que las características de cada registro se interpreten como entrada para nuestro
# modelo.
assembler = VectorAssembler(inputCols=[c for c in df.columns if c not in ['transaction_id', 'timestamp', 'is_fraud']], outputCol="features")
df = assembler.transform(df)

# Creamos el MinMaxScaler
min_max_scaler = MinMaxScaler(inputCol="features", outputCol="s_features")
min_max_scaler_model = min_max_scaler.fit(df)
df = min_max_scaler_model.transform(df)


In [9]:
# Seleccionar la variable objetivo y las características.
df = df.select(['transaction_id', 's_features', 'is_fraud'])
df.show()

+--------------------+--------------------+--------+
|      transaction_id|          s_features|is_fraud|
+--------------------+--------------------+--------+
|448eab14ab3be1100...|[0.02253195162451...|       0|
|b3366af5dd24da698...|[0.14005089680930...|       0|
|f64640528d1bc4d50...|[0.01410246158043...|       0|
|d69b16d989f997c4e...|[0.00536251040750...|       0|
|82f6b02e113e27b6a...|[0.00713590200717...|       0|
|2b58a1b1a4353a944...|[0.05484813276446...|       0|
|c655ea515a0ecd105...|[0.03189752901165...|       0|
|8196944079c991dea...|[0.00893281338934...|       0|
|924aaf66f8fa0fc12...|[0.00470395649781...|       0|
|eed3cd952acca5e88...|[0.04322936021487...|       0|
|a839296313a9dce64...|[0.07514100109602...|       0|
|e036495d9ed4b0cce...|[0.01175518728802...|       0|
|84b31c379b7bbdc7d...|[9.97238777535785...|       0|
|844c813a2822c32e1...|[9.31383386566441...|       0|
|7bb47998ec31c4aeb...|[0.13660289669641...|       0|
|4945e4cdd8d706739...|[0.02163819988992...|   

# Justificación

Los árboles de desición son bien conocidos por implementarse en problemás de clasificación. Cada nodo de la estructura que los compone analiza cada caracteristica de las muestras para "tomar" una desición. Es en este sentido por el que se opta por un RandomForest, una generalización del modelo que permite una mejor robustes y resistencia al sobreajuste.

In [10]:
from pyspark.ml.classification import RandomForestClassifier

# Dividimos los datos en conjuntos de entrenamiento y prueba.
train_df, test_df = df.randomSplit([0.7, 0.3], seed=100) #le asignamos un valor a seed para que siempre obtengamos la misma 
                                                        # separación de datos

# Entrenamos el modelo
rf = RandomForestClassifier(labelCol="is_fraud", featuresCol="s_features", numTrees=100) 
model = rf.fit(train_df) # pySpark nos ayuda con la función fit para el entrenamiento automático del modelo

# Realizamos las  predicciones
predictions = model.transform(test_df)
predictions.select("transaction_id","s_features", "is_fraud", "prediction", "probability").show()


+--------------------+--------------------+--------+----------+--------------------+
|      transaction_id|          s_features|is_fraud|prediction|         probability|
+--------------------+--------------------+--------+----------+--------------------+
|4945e4cdd8d706739...|[0.02163819988992...|       0|       0.0|[0.75089884006146...|
|924aaf66f8fa0fc12...|[0.00470395649781...|       0|       0.0|[0.94578088742521...|
|a839296313a9dce64...|[0.07514100109602...|       0|       0.0|[0.74244161706726...|
|c655ea515a0ecd105...|[0.03189752901165...|       0|       0.0|[0.85121720901494...|
|d69b16d989f997c4e...|[0.00536251040750...|       0|       0.0|[0.90095264002227...|
|3a5828ab712552202...|[9.31383386566441...|       0|       0.0|[0.94328070632250...|
|3a5bded1fad7cb8ae...|[0.03101788914656...|       0|       0.0|[0.94084160282494...|
|b606422a35eeb682b...|[4.70395649781030...|       0|       0.0|[0.92633196833539...|
|bb18dc7a39a08b2d5...|[4.18652128305117...|       0|       0.0|[0

# Testing de modelo
Se incluyen medidas tipicas para la evaluación del modelo.

In [11]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

# ROC AUC Score
evaluator = BinaryClassificationEvaluator(labelCol="is_fraud", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
roc_auc = evaluator.evaluate(predictions)
print(f'ROC AUC Score: {roc_auc}')

# Matriz de confusión
predictions.groupBy("is_fraud", "prediction").count().show()

# Precision y Recall
evaluator_precision = MulticlassClassificationEvaluator(labelCol="is_fraud", predictionCol="prediction", metricName="precisionByLabel")
evaluator_recall = MulticlassClassificationEvaluator(labelCol="is_fraud", predictionCol="prediction", metricName="recallByLabel")
precision = evaluator_precision.evaluate(predictions)
recall = evaluator_recall.evaluate(predictions)
print(f'Precision: {precision}')
print(f'Recall: {recall}')

# F1 Score
evaluator_f1 = MulticlassClassificationEvaluator(labelCol="is_fraud", predictionCol="prediction", metricName="f1")
f1_score = evaluator_f1.evaluate(predictions)
print(f'F1 Score: {f1_score}')

# Accuracy
evaluator_accuracy = MulticlassClassificationEvaluator(labelCol="is_fraud", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator_accuracy.evaluate(predictions)
print(f'Accuracy: {accuracy}')

ROC AUC Score: 0.9687326222132349
+--------+----------+-----+
|is_fraud|prediction|count|
+--------+----------+-----+
|       0|       0.0|  129|
|       0|       1.0|    2|
|       1|       0.0|   19|
|       1|       1.0|  132|
+--------+----------+-----+

Precision: 0.8716216216216216
Recall: 0.9847328244274809
F1 Score: 0.9255796778608611
Accuracy: 0.925531914893617


In [12]:
# Convertir predicciones a un DataFrame de pandas
predictions_pd = predictions.select("transaction_id", "probability").toPandas()
predictions_pd['score'] = predictions_pd['probability'].apply(lambda x: x[1])
#El score representa así, la posibilidad de que la transacción sea fraudulenta
predictions_pd = predictions_pd.drop('probability', axis=1)

# Guardar como CSV
predictions_pd.to_csv('predicciones_fraude.csv', index=False)
